# Exercise 5: Pandas
# Duy Tommy Tran

#### Importing library

In [1]:
import numpy as np
import pandas as pd

In [2]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

#### 1.

Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [3]:
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)
#data

In [4]:
def get_DataFrame(N):
    return data[:N]

df = get_DataFrame(10001)
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
9996,1,0,60,3869200267,701,5
9997,1,0,59,3869200267,707,23
9998,1,0,63,3869200267,706,15
9999,1,0,49,3869200267,777,13


#### 2.

Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [5]:
BX = data['BX_COUNTER']
BX_max = np.max(BX)

print(f'The number of BX in an ORBIT is: {BX_max}')

The number of BX in an ORBIT is: 3563


#### 3.

Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [6]:
data['ABS_TIME'] = data['TDC_MEAS']*25/30 + data['BX_COUNTER']*25 + data['ORBIT_CNT']*25*(BX_max+1)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,1,0,63,3869200167,2553,28,3.447457e+14
3,1,0,64,3869200167,2558,19,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,3.447467e+14
1310716,1,1,4,3869211171,763,11,3.447467e+14
1310717,1,0,64,3869211171,764,0,3.447467e+14
1310718,1,0,139,3869211171,769,0,3.447467e+14


#### 4. 

Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [7]:
abs_time = data['ABS_TIME']
T = abs_time.iloc[-1] - abs_time.iloc[0]
T

980416093.3125

#### 5.

Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [8]:
noisy = data.groupby('TDC_CHANNEL').size().sort_values()

print('The 3 TDC channels with most counts and their corresponding counts: \n') 
print(noisy[-3:])

The 3 TDC channels with most counts and their corresponding counts: 

TDC_CHANNEL
63      64642
64      66020
139    108059
dtype: int64


#### 6. 
Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [9]:
orbits = data['ORBIT_CNT']
num_orbits = len(set(orbits))

print(f'The number of non-empty orbits are: {num_orbits}')

The number of non-empty orbits are: 11001


#### 7.
Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [10]:
orbits_139 = data[data['TDC_CHANNEL'] == 139]['TDC_CHANNEL']

print(f'The number of unique orbits from TDC_CHANNEL=139 are: {len(orbits_139)}')

The number of unique orbits from TDC_CHANNEL=139 are: 108059


#### 8.
Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [11]:
s0 = data[data['FPGA'] == 0]
s1 = data[data['FPGA'] == 1]

count0 = s0.groupby('TDC_CHANNEL').count()
count1 = s1.groupby('TDC_CHANNEL').count()